In [1]:
# Read Data From Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install dipy==1.1.1
!pip install SimpleITK==2.0.0
!pip install nibabel==3.1.1

In [ ]:
cd 'drive/MyDrive/Thesis_Code/'

In [ ]:
#Preprocess TrainSet
!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/train' -i /content/drive/MyDrive/Thesis_Code/Data/Train -s ID01_01_ISBR ID01_02_ISBR ID01_03_ISBR ID01_04_ISBR ID02_01_ISBR ID02_02_ISBR ID02_03_ISBR ID02_04_ISBR ID03_02_ISBR ID03_03_ISBR ID04_01_ISBR ID04_02_ISBR ID04_03_ISBR ID05_01_ISBR ID05_02_ISBR ID05_03_ISBR --input_image_suffix_1 _FLAIR.nii.gz --input_image_suffix_2 _T1.nii.gz --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz --label_suffix_1 _MASK_1.nii.gz --label_suffix_2 _MASK_2.nii.gz -f ISBR_DATA_Train.json --n_classes 2 --zooms 1. 1. 1

In [ ]:
#Preprocess ValidationSet
!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/validation' -i /content/drive/MyDrive/Thesis_Code/Data/Validation -s ID03_01_ISBR ID03_04_ISBR ID04_04_ISBR ID05_04_ISBR --input_image_suffix_1 _FLAIR.nii.gz --input_image_suffix_2 _T1.nii.gz --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz --label_suffix_1 _MASK_1.nii.gz --label_suffix_2 _MASK_2.nii.gz -f ISBR_DATA_Validation.json --n_classes 2 --zooms 1. 1. 1

In [ ]:
from my_models import TwoPathModel

from torch.utils.data import DataLoader
from my_datasets.ISBI_dataset import ISBI_DataSet

from my_utils.expriments import train_model
from my_utils.expriments import test_model

from my_losses import FocalTverskyLoss2

import torch.nn as nn
import torch.optim as opt
import torch

import albumentations as A

In [ ]:
dataset_path = {
    'train': r"./Images_CLAHE_Smooth2/train",
    'validation' : r"./Images_CLAHE_Smooth2/validation",
    'test' : r"./Images_CLAHE_Smooth2/test"
}

result_path = './Results'




num_patch = 15
lesion_pr = 0.6
batch_size = 3


max_epoch = 100
n_classes = 2
in_channels = 2
model_name = 'test'
learning_rate = 0.0008

input_shape = [80,80,80]
output_shape = [60,60,60]
num_modality = 1


In [ ]:
my_datasets = {x: ISBI_DataSet(dataset_path = dataset_path[x],
                      input_shape = input_shape,
                      output_shape = output_shape,
                      for_data = x,
                      num_patch = num_patch,
                      two_label = True,
                      num_modality = num_modality,
                      multi_task=True                      
                      )
                                  
                  for x in ['train', 'test', 'validation']}

my_dataloaders = {x: DataLoader(my_datasets[x], 
                                batch_size= batch_size if x=='train' else 1,
                                shuffle=True if x=='train' else None)
                                  
                  for x in ['train', 'test', 'validation']}


num_data = {'train':len(my_datasets['train']), 'validation':len(my_datasets['validation']), 'test':len(my_datasets['test'])}

In [ ]:
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TwoPathModel(in_channels=in_channels, n_classes=n_classes, num_modality=num_modality)

optimizer = opt.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0005)
scheduler = opt.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = FocalTverskyLoss2()


The model is single modal


In [ ]:
model = train_model(model=model,
                    dataloaders = my_dataloaders,
                    criterion = criterion,
                    optimizer = optimizer,
                    device = device,
                    num_data = num_data,
                    input_shape = input_shape,
                    output_shape = output_shape,
                    n_classes = n_classes,
                    scheduler = scheduler,
                    model_name = model_name,
                    max_epoch = max_epoch)

In [ ]:
# Preprocess Test Set #idx
!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_01 -s test01_01 test01_02 test01_03 test01_04 --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_02 -s test02_01 test02_02 test02_03 test02_04 test02_05 --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_03 -s test03_01 test03_02 test03_03 test03_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_04 -s test04_01 test04_02 test04_03 test04_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_05 -s test05_01 test05_02 test05_03 test05_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_06 -s test06_01 test06_02 test06_03 test06_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_07 -s test07_01 test07_02 test07_03 test07_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_08 -s test08_01 test08_02 test08_03 test08_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_09 -s test09_01 test09_02 test09_03 test09_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_10 -s test10_01 test10_02 test10_03 test10_04 test10_05 test10_06  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_11 -s test11_01 test11_02 test11_03 test11_04 test11_05 --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_12 -s test12_01 test12_02 test12_03 test12_04  --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_13 -s test13_01 test13_02 test13_03 test13_04 --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

!python -W ignore my_utils/preprocess.py -c 'Images_CLAHE_Smooth2/test' -i /content/drive/MyDrive/Thesis_Code/Data/Test/test_14 -s test14_01 test14_02 test14_03 test14_04 test14_05 --input_image_suffix_1 _flair_pp.nii --input_image_suffix_2 _mprage_pp.nii --output_image_suffix_1 _FLAIR_Pre.nii.gz --output_image_suffix_2 _T1_Pre.nii.gz  -f ISBR_DATA_Test.json --n_classes 2 --zooms 1. 1. 1

In [ ]:
state_dict = torch.load(f'Checkpoint_Files/{model_name}_checkpoint.tar', map_location=device)['model_state_dict']
print(f'Files from Checkpoint_Files/{model_name}_checkpoint.tar is loaded')
model.load_state_dict(state_dict)

model = model.to(device)

Files from Checkpoint_Files/SinglePath_checkpoint.tar is loaded


In [ ]:
test_model(model = model,
           dataloder = my_dataloaders['test'],
           num_data = num_data['test'],
           n_classes = n_classes,
           input_shape = input_shape,
           output_shape = output_shape,
           device = device,
           image_suffix = f'{model_name}.nii.gz',
           result_path = result_path)

In [ ]:
import torch

dummy_input = torch.randn(1, 2, 80, 80, 80, device=device, dtype=torch.float32)

input_names = [ "actual_input_1" ] 
output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, input_names=input_names, output_names=output_names)